In [1]:
import pandas as pd
import numpy as np
import math
from tqdm import tqdm, trange
import random
from random import sample
import math
from collections import defaultdict  
import matplotlib.pyplot as plt
import seaborn as sns
plt.rc('font', family='SimHei', size=13)

import os,gc,re,warnings,sys
warnings.filterwarnings("ignore")

from tqdm import tqdm

In [27]:
usercf_df = pd.read_csv('xu_itemcf.csv', header=None)
itemcf_df = pd.read_csv('deng_itemcf.csv', header=None)
usercf_df = pd.DataFrame(usercf_df, dtype=int)
itemcf_df = pd.DataFrame(itemcf_df, dtype=int)



In [28]:
uer_loc_dict = {}
for user in tqdm(usercf_df[0].unique()):
    uer_loc_dict.setdefault(user, {})
    for loc, item in enumerate(usercf_df[usercf_df[0] == user].iloc[0][1:].tolist()):
        uer_loc_dict[user].setdefault(item, 0)
        uer_loc_dict[user][item] = loc

100%|███████████████████████████████████████████████████████████████████████████| 12081/12081 [00:17<00:00, 694.64it/s]


In [29]:
item_loc_dict = {}
for user in tqdm(itemcf_df[0].unique()):
    item_loc_dict.setdefault(user, {})
    for loc, item in enumerate(itemcf_df[itemcf_df[0] == user].iloc[0][1:].tolist()):
        item_loc_dict[user].setdefault(item, 0)
        item_loc_dict[user][item] = loc


100%|███████████████████████████████████████████████████████████████████████████| 12081/12081 [00:18<00:00, 662.27it/s]


In [30]:
user_item_rank = {}
for user in tqdm(usercf_df[0].unique()):
    same_items = list(
        set(usercf_df[usercf_df[0] == user].iloc[0][1:]) & set(itemcf_df[itemcf_df[0] == user].iloc[0][1:]))
    user_item_rank.setdefault(user, {})
    for item in same_items:
        user_item_rank[user].setdefault(item, 0)
        user_item_rank[user][item] = uer_loc_dict[user][item] + item_loc_dict[user][item]
    # 对于两个集合交集之外的item，分别加上50
    other1_items = list(set(itemcf_df[itemcf_df[0] == user].iloc[0][1:]) - set(same_items))
    other2_items = list(set(usercf_df[usercf_df[0] == user].iloc[0][1:]) - set(same_items))
    for item in other1_items:
        user_item_rank[user].setdefault(item, 0)
        user_item_rank[user][item] = item_loc_dict[user][item] + 50
    for item in other2_items:
        user_item_rank[user].setdefault(item, 0)
        user_item_rank[user][item] = uer_loc_dict[user][item] + 50


100%|███████████████████████████████████████████████████████████████████████████| 12081/12081 [01:16<00:00, 158.71it/s]


In [31]:
sorted_item_sim = {k: list(sorted(v.items(), key=lambda x: int(x[1]), reverse=False)) for k, v in user_item_rank.items()}

result = []
for user in sorted_item_sim:
    recall_items = []
    recall_items.append(user)
    for item, wij in sorted_item_sim[user][0:50]:
        recall_items.append(item)
    result.append(recall_items)

In [32]:
result = pd.DataFrame(result)

In [33]:
result

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,1,32360,100116,94147,64538,67733,86677,46297,16734,92349,...,84994,19438,14665,40040,62491,97393,13845,102700,20165,51719
1,2,18104,82469,55815,4612,7136,26837,10612,26953,92469,...,72295,98102,44198,26436,23057,42713,48538,92198,91190,38098
2,3,110798,107129,47622,87420,17287,87807,25959,48446,37485,...,105381,20208,93353,52860,52697,69720,59040,56024,77999,85203
3,11,26711,40801,10528,21517,85475,77847,59255,59376,24148,...,65207,100117,59818,70957,97305,53384,21974,5510,43306,31527
4,13,97782,23554,50875,21082,64789,29815,22947,41282,104742,...,100043,38342,42109,45277,76469,62762,111576,67726,39021,44247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12076,35426,101826,89709,17282,63340,1760,22636,60733,73111,29514,...,77602,76465,112044,37713,42688,41189,713,17691,44279,37233
12077,35434,43017,60349,94200,81022,64550,70705,76806,67607,59311,...,70691,22404,89798,28015,49283,90661,32601,45578,58165,63321
12078,35435,31115,91026,34249,23634,32280,99084,44684,65742,69642,...,20034,27011,17632,23874,24933,42541,4229,15856,20051,41807
12079,35436,29411,87047,92533,15960,68704,115973,98772,7387,63103,...,60163,47520,67609,69746,10116,100808,98645,68478,18556,68716


In [34]:
result.to_csv('submission_add.csv',index=False)
pd.read_csv('submission_add.csv')

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,1,32360,100116,94147,64538,67733,86677,46297,16734,92349,...,84994,19438,14665,40040,62491,97393,13845,102700,20165,51719
1,2,18104,82469,55815,4612,7136,26837,10612,26953,92469,...,72295,98102,44198,26436,23057,42713,48538,92198,91190,38098
2,3,110798,107129,47622,87420,17287,87807,25959,48446,37485,...,105381,20208,93353,52860,52697,69720,59040,56024,77999,85203
3,11,26711,40801,10528,21517,85475,77847,59255,59376,24148,...,65207,100117,59818,70957,97305,53384,21974,5510,43306,31527
4,13,97782,23554,50875,21082,64789,29815,22947,41282,104742,...,100043,38342,42109,45277,76469,62762,111576,67726,39021,44247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12076,35426,101826,89709,17282,63340,1760,22636,60733,73111,29514,...,77602,76465,112044,37713,42688,41189,713,17691,44279,37233
12077,35434,43017,60349,94200,81022,64550,70705,76806,67607,59311,...,70691,22404,89798,28015,49283,90661,32601,45578,58165,63321
12078,35435,31115,91026,34249,23634,32280,99084,44684,65742,69642,...,20034,27011,17632,23874,24933,42541,4229,15856,20051,41807
12079,35436,29411,87047,92533,15960,68704,115973,98772,7387,63103,...,60163,47520,67609,69746,10116,100808,98645,68478,18556,68716
